# Day 20: Particle Swarm

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (`p`), velocity (`v`), and acceleration (`a`), each in the format `<X,Y,Z>`.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

- Increase the `X` velocity by the `X` acceleration.
- Increase the `Y` velocity by the `Y` acceleration.
- Increase the `Z` velocity by the `Z` acceleration.
- Increase the `X` position by the `X` velocity.
- Increase the `Y` position by the `Y` velocity.
- Increase the `Z` position by the `Z` velocity.

Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position `<0,0,0>` in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's `X`, `Y`, and `Z` position.

In [1]:
import numpy as np, copy


def parse_var(s):
    name = s[0]
    assert s[1:3] == '=<' and s[-1] == '>'
    return name, np.array([int(x) for x in s[3:-1].split(',')], dtype=int)


def parse(s):
    lines = s.strip().splitlines()
    particles = []
    for line in lines:
        particle = {}
        for var in line.split(', '):
            k, v = parse_var(var)
            particle[k] = v
        particles.append(particle)
    return particles


def step(particles):
    particles = copy.deepcopy(particles)
    for p in particles:
        p['v'] += p['a']
        p['p'] += p['v']
    return particles

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). 

Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

In [2]:
def same(ps, qs):
    if len(ps) != len(qs):
        return False
    for p, q in zip(ps, qs):
        p = {k: tuple(v) for k, v in p.items()}
        q = {k: tuple(v) for k, v in q.items()}
        if p != q:
            return False
    return True


#    -4 -3 -2 -1  0  1  2  3  4
#                         (0)(1)
example = parse("""
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
""")

#    -4 -3 -2 -1  0  1  2  3  4
#                      (1)   (0)
expected = parse("""
p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>
""")
particles = step(example)
assert same(particles, expected)

#    -4 -3 -2 -1  0  1  2  3  4
#          (1)               (0)
expected = parse("""
p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>
""")
particles = step(particles)
assert same(particles, expected)

#    -4 -3 -2 -1  0  1  2  3  4
#                         (0)
expected = parse("""
p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>
""")
particles = step(particles)
assert same(particles, expected)

At this point, particle 1 will never be closer to `<0,0,0>` than particle 0, and so, in the long run, particle 0 will stay closest.

In [3]:
def manhattan(v):
    return np.sum(np.abs(v))


def closest_in_long_run(particles):
    # find those with smallest acceleration
    min_a = np.min([manhattan(p['a']) for p in particles])
    argmin_a = [
        i for i, p in enumerate(particles) if manhattan(p['a']) == min_a
    ]

    # among those, find those with smallest velocity
    min_v = np.min([manhattan(particles[i]['v']) for i in argmin_a])
    argmin_v = [i for i in argmin_a if manhattan(particles[i]['v']) == min_v]

    # assume there is only a single such particle
    assert len(argmin_v) == 1
    return argmin_v[0]


assert closest_in_long_run(example) == 0

Which particle will stay closest to position `<0,0,0>` in the long term

In [4]:
puzzle = parse(open('20.input').read())

closest_in_long_run(puzzle)

243

## Part Two

To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

In [5]:
from functools import reduce
from collections import defaultdict

ANY = 'ANY'


def solve(a, b, c):
    """
    return solutions to quadratic equation A*t^2 + B*t + C == 0.
    returns ANY if any t is a solution (i.e., if A=B=C=0).
    """
    # quadratic
    if a:
        if b**2 < 4 * a * c:
            return set()
        p, q = b / a, c / a
        return {-p / 2 + np.sqrt(p**2 / 4 - q), -p / 2 - np.sqrt(p**2 / 4 - q)}

    # linear
    if b:
        return {-c / b}

    # constant
    return set() if c else ANY


def integers(ts):
    """return integer solutions in given set (or ANY when given ANY)"""
    DIGITS = 8
    if ts == ANY:
        return ANY
    ts = {round(t, DIGITS) for t in ts}
    return {int(t) for t in ts if t == int(t)}


def intersect(a, b):
    return b if a == ANY else a if b == ANY else a & b


def intersect_all(sets):
    return reduce(intersect, sets)


assert intersect_all([
    integers(solve(1, 0, -4)),
    integers(solve(0, 0, 0)),
    integers(solve(0, 1, -2)),
]) == {2}


def collisions(particles):
    """return all potential collisions at integer times t >= 0, indexed by the collision time"""
    result = defaultdict(set)
    for i, p in enumerate(particles):
        for j, q in enumerate(particles):
            if i >= j: continue
            # solve (a-a')*t**2 + ((a-a')+2*(v-v'))*t + 2*(p-p') == 0
            a = p['a'] - q['a']
            b = (p['a'] - q['a']) + 2 * (p['v'] - q['v'])
            c = 2 * (p['p'] - q['p'])
            ts = intersect_all([
                integers(solve(a[0], b[0], c[0])),
                integers(solve(a[1], b[1], c[1])),
                integers(solve(a[2], b[2], c[2])),
            ])

            # add collision if it happens at nonnegative time
            for t in ts:
                if t >= 0:
                    result[t].add(i)
                    result[t].add(j)
    return result

For example:

In [6]:
example = parse("""
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>
""")

assert collisions(example) == {2: {0, 1, 2}}

Indeed:

```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

In [7]:
def resolve_collisions(particles):
    cols = collisions(particles)
    alive = set(range(len(particles)))
    while cols:
        # resolve earliest collision
        t = min(cols)
        alive -= cols[t]
        print('t=%d: %r removed, %d particles remaining' % (t, cols[t],
                                                            len(alive)))

        # remove those particles from all later collisions
        for t in list(cols.keys()):
            rest = cols[t] & alive
            if len(rest) > 1:
                cols[t] = rest
            else:
                del cols[t]
    return alive


assert resolve_collisions(example) == {3}

t=2: {0, 1, 2} removed, 1 particles remaining


How many particles are left after all collisions are resolved?

In [8]:
%%time
len(resolve_collisions(puzzle))

t=10: {115, 116, 117, 118, 119, 120, 121} removed, 993 particles remaining
t=12: {37, 38, 39, 40, 41, 42, 43, 44, 45, 46} removed, 983 particles remaining
t=13: {87, 88, 89, 90, 91, 92} removed, 977 particles remaining
t=14: {153, 311, 152, 93, 150, 151, 312, 313, 314, 315, 316, 317, 94} removed, 964 particles remaining
t=15: {263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 327, 329, 328, 331, 332, 333, 330} removed, 947 particles remaining
t=16: {0, 1, 2, 3, 4, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23} removed, 932 particles remaining
t=17: {177, 178, 179, 180, 181, 182, 183, 184, 185, 186} removed, 922 particles remaining
t=18: {5, 6, 7, 8, 9, 10, 11, 12, 13, 239, 240} removed, 911 particles remaining
t=19: {251, 252} removed, 909 particles remaining
t=20: {157, 158, 159, 32, 33, 34, 35, 36, 160, 161, 162, 163, 164, 165, 79, 80, 81, 82, 83, 84, 85, 86, 230, 231, 232, 233, 234, 235, 236, 237, 238} removed, 878 particles remaining
t=21: {166, 167, 168, 169, 170, 171, 172, 173, 174, 175

648